In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
csv_file_path = '/content/final_data.csv'
data = pd.read_csv(csv_file_path)

# Split the data into a 60:40 ratio
train_data, test_data = train_test_split(data, test_size=0.5, random_state=42)

# Save the split data into new CSV files
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the testing data CSV file into a DataFrame
test_data = pd.read_csv('test_data.csv')

# Split the testing data into labeled (50%) and unlabeled (50%) parts
labeled_data, unlabeled_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Save the labeled and unlabeled data into separate CSV files
labeled_data.to_csv('labeled_data.csv', index=False)
unlabeled_data.drop(columns=['Label'], inplace=True)  # Drop the label column
unlabeled_data.to_csv('unlabeled_data.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import re
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Dense, Concatenate, BatchNormalization



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re

# Load data
data = pd.read_csv('/content/train_data.csv')

# Extract URL strings and labels
X = data[['URL','Content-Type','Payload']]
y = data['Label']


X = X.astype(str)
print(type(X))
print(len(X))
Y = y.astype(str)
print(type(Y))
print(len(Y))
special_characters = ["'", '"', '<', '>', '+', '-', '_', '*', '=', '{', '}', '(', ')', '[', ']', '∼', '/',
                      '\\', '#', ':', ';', '?', '!', '&', '@']


<class 'pandas.core.frame.DataFrame'>
71590
<class 'pandas.core.series.Series'>
71590


In [ ]:

tokenized_payloads = []

def tokenize_and_segment(X):
    for payload in X:
        segmented_payload = []
        current_segment = ""

        for char in payload:
            if char in special_characters:
                if current_segment:
                    segmented_payload.append(current_segment)
                current_segment = ""
                segmented_payload.append(char)
            else:
                current_segment += char

        if current_segment:
            segmented_payload.append(current_segment)

        tokenized_payloads.append(segmented_payload)

    return tokenized_payloads

# Tokenize the payloads
tokenized_payloads = tokenize_and_segment(X['Payload'])
vocabulary = {}
index = 0
for sample in tokenized_payloads:
    for token in sample:
        if token not in vocabulary:
            vocabulary[token] = index
            index += 1


In [ ]:

# Step 4: Sequence Padding/Truncation
max_sequence_length = 498 # Adjust this value based on your requirements

# Convert tokens to indices based on vocabulary
sequences = [[vocabulary[token] for token in sample] for sample in tokenized_payloads]

# Pad sequences to a maximum length/
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Print the padded sequences
print(len(sequences))
print(padded_sequences)


71590
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  1   2   3 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [107   5   2 ...   0   0   0]]


In [ ]:

# Initialize a Tokenizer
tokenizer = Tokenizer(filters=special_characters, oov_token='<OOV>')

# Fit the tokenizer on payload data to build the vocabulary
tokenizer.fit_on_texts(X['Payload'])

# Convert text to sequences of indices
sequences = tokenizer.texts_to_sequences(X['Payload'])

# Pad sequences to a maximum length
max_sequence_length = 498
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Print the padded sequences
print(len(sequences))
print(len(padded_sequences))


71590
71590


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Dense, Concatenate, BatchNormalization
# from tensorflow.keras.utils import to_categorical

# # Load data
# data = pd.read_csv('/content/final_data.csv',encoding='UTF-8')

# # Extract URL strings and labels
# X = data[['URL', 'Content-Type', 'Payload']]
# y = data['Label']



In [ ]:
X = X.astype(str)
print(type(X))
print(len(X))
Y = y.astype(str)
print(type(Y))
print(len(Y))

<class 'pandas.core.frame.DataFrame'>
71590
<class 'pandas.core.series.Series'>
71590


In [ ]:
# Tokenize and pad sequences
max_sequence_length = 100
tokenizer = Tokenizer(filters=["'", '"', '<', '>', '+', '-', '_', '*', '=', '{', '}', '(', ')', '[', ']', '∼', '/',
                      '\\', '#', ':', ';', '?', '!', '&', '@'], oov_token='<OOV>')
tokenizer.fit_on_texts(X['Payload'])
sequences = tokenizer.texts_to_sequences(X['Payload'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Convert labels to one-hot encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)
one_hot_labels = to_categorical(encoded_labels)



In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)

In [ ]:
# TextCNN model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)

conv_layers = []
filter_sizes = [3, 4, 5]
for filter_size in filter_sizes:
    conv = Conv1D(filters=64, kernel_size=filter_size, activation='relu')(embedding_layer)
    pool = GlobalMaxPooling1D()(conv)
    conv_layers.append(pool)

concatenated = Concatenate()(conv_layers)
dropout = Dropout(0.5)(concatenated)
dense = Dense(64, activation='relu')(dropout)
output_layer = Dense(len(label_encoder.classes_), activation='softmax')(dense)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 100, 128)     6053888     ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 98, 64)       24640       ['embedding_1[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 97, 64)       32832       ['embedding_1[0][0]']            
                                                                                            

In [ ]:
from keras.models import load_model
# Assuming you have a trained model stored in the 'model' variable
model.save('trained_model.h5')


In [ ]:
import numpy as np
import random
import csv

def apply_keyword_avoidance_perturbation(original_sample, keyword_library):
    def randomize_number(n):
        return str(np.random.randint(10))

    def randomize_letter(c):
        if c.isalpha():
            return chr(np.random.randint(ord('a'), ord('z') + 1))
        return c

    perturbed_words = []
    for word in original_sample:
        if word in keyword_library:
            perturbed_words.append(word)
        else:
            special_characters = ['\'', '"', '<', '>', '+', '-', '_', '*', '=', '{', '}', '(', ')', '[', ']', '∼', '/',
                                  '\\', '#', ':', ';', '?', '!', '&', '@', '%']
            if any(char in special_characters for char in word):
                perturbed_words.append(word)
            else:
                perturbed_word = ''.join(randomize_number(c) if c.isdigit() else randomize_letter(c) for c in word)
                perturbed_words.append(perturbed_word)

    return perturbed_words

# Define the keyword library
keyword_library = {"login", "password", "malicious", "attack", "exploit", "hacker", "virus",
                   "SELECT", "UNION", "AND", "http", "script"}

# Example usage of data augmentation with payload
tokenized_payloads = tokenized_payloads  # Your list of tokenized payloads

augmented_payloads = []
for original_payload in tokenized_payloads:
    augmented_payload = apply_keyword_avoidance_perturbation(original_payload, keyword_library)
    augmented_payloads.append(augmented_payload)

# Save the results to a CSV file
csv_file = "augmented_payloads.csv"

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Original Payload", "Augmented Payload"])  # Write header

    for original_payload, augmented_payload in zip(tokenized_payloads, augmented_payloads):
        writer.writerow([original_payload, augmented_payload])

print(f"Results saved to {csv_file}")


In [ ]:

url = data['URL'].tolist()

In [ ]:
import numpy as np
import random
import pandas as pd
import csv

def apply_keyword_avoidance_perturbation(original_url, keyword_library):
    def randomize_number(n):
        return str(np.random.randint(10))

    def randomize_letter(c):
        if c.isalpha():
            return chr(np.random.randint(ord('a'), ord('z') + 1))
        return c

    components = re.split(r'(/|=|\?|&|\.)', original_url)  # Split using regex
    perturbed_components = []

    for component in components:
        if component in ('/', '=', '.', "&", "?"):
            perturbed_components.append(component)
            continue

        if any(keyword in component for keyword in keyword_library):
            perturbed_components.append(component)
        else:
            perturbed_component = ''.join(randomize_number(c) if c.isdigit() else randomize_letter(c) for c in component)
            perturbed_components.append(perturbed_component)

    perturbed_url = ''.join(perturbed_components)
    return perturbed_url

# Example DataFrame with URLs
# data = pd.read_csv('your_data.csv')  # Replace with the actual file name
url = data['URL'].tolist()

# Define the keyword library
keyword_library = {"login", "user", "password", "malicious", "attack", "exploit", "hacker", "virus",
                   "com", "co", "cn", "in", "http", "php"}

# Apply data augmentation to each URL in the DataFrame
augmented_urls = []
for original_url in url:
    if isinstance(original_url, str):  # Check for NaN values
        augmented_url = apply_keyword_avoidance_perturbation(original_url, keyword_library)
        augmented_urls.append(augmented_url)

# Save the results to a CSV file
csv_file = "augmented_urls.csv"

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Original url", "Augmented url"])  # Write header

    for original_url, augmented_url in zip(url, augmented_urls):
        if isinstance(original_url, str):
            writer.writerow([original_url, augmented_url])

print(f"Results saved to {csv_file}")


Results saved to augmented_urls.csv


In [ ]:
X = data[['URL', 'Content-Type', 'Payload']]
url_list = X['URL'].tolist()

print(url_list[:5])

['www.google-analytics.com/analytics.js', 'ipv6.msftconnecttest.com/connecttest.txt', 'www.netdania.com/jolokia/list', 'www.netdania.com/', 'www.netdania.com/']


In [ ]:
import random
import csv

def perform_data_augment(payload):
    # Randomly choose one augmentation technique
    augmentation_technique = random.choice(["insertion", "deletion", "substitution"])

    if augmentation_technique == "insertion":
        # Perform insertion augmentation
        augmented_payload = payload + random.choice(["ADD", "<script>", "</script>", "SELECT"])
    elif augmentation_technique == "deletion":
        # Perform deletion augmentation
        augmented_payload = payload[:-1]
    else:
        # Perform substitution augmentation
        random_index = random.randint(0, len(payload)-1)
        augmented_payload = payload[:random_index]  + payload[random_index+1:]

    return augmented_payload

def generate_augment_csv(input_csv_path, output_csv_path):
    with open(input_csv_path, 'r') as input_file, open(output_csv_path, 'w', newline='') as output_file:
        reader = csv.reader(input_file)
        writer = csv.writer(output_file)

        header = next(reader)
        writer.writerow(header + ["Augmented Payload"])

        for row in reader:
            payload = row[0]
            augmented_payload = perform_data_augment(payload)
            writer.writerow(row + [augmented_payload])

if __name__ == "__main__":
    input_csv_path = "target.csv"  # Provide the path to your input CSV file
    GDTSL = "GDTSL.csv"  # Provide the desired path for the output CSV file
    generate_augment_csv(input_csv_path, GDTSL)


In [ ]:
import random
import csv

def perform_data_augment(payload):
    # Randomly choose one augmentation technique
    augmentation_technique = random.choice(["insertion", "deletion", "substitution"])

    if augmentation_technique == "insertion":
        # Perform insertion augmentation
        augmented_payload = payload + random.choice(["ADD", "<script>", "</script>", "SELECT"])
    elif augmentation_technique == "deletion":
        # Perform deletion augmentation
        augmented_payload = payload[:-1]
    else:
        # Perform substitution augmentation
        random_index = random.choice(range(len(payload)))
        augmented_payload = payload[:random_index]  + payload[random_index+1:]

    return augmented_payload

def generate_augment_csv(input_csv_path, output_csv_path):
    with open(input_csv_path, 'r') as input_file, open(output_csv_path, 'w', newline='') as output_file:
        reader = csv.reader(input_file)
        writer = csv.writer(output_file)

        header = next(reader)
        writer.writerow(header + ["Augmented Payload"])

        for row in reader:
            payload = row[0]
            augmented_payload = perform_data_augment(payload)
            writer.writerow(row + [augmented_payload])

if __name__ == "__main__":
    input_csv_path = "/content/unlabelled.csv"  # Provide the path to your input CSV file
    GDTSL = "GDTSU.csv"  # Provide the desired path for the output CSV file
    generate_augment_csv(input_csv_path, GDTSL)


In [ ]:
DTSL = pd.read_csv('/content/target.csv')
DTSU = pd.read_csv('/content/unlabelled.csv')

In [ ]:
GDTSL = pd.read_csv('/content/GDTSL.csv')
GDTSU = pd.read_csv('/content/GDTSU.csv')

In [ ]:
# import pandas as pd

# # Assuming DTSU and GDTSL are DataFrame objects
# shape = DTSU.shape
# DTSU_array = DTSU.to_numpy().reshape(shape)
# GDTSU_array = GDTSU.to_numpy().reshape(shape)

# # Create new DataFrame objects with reshaped arrays
# DTSU = pd.DataFrame(DTSU_array, columns=DTSU.columns, index=DTSU.index)
# GDTSU = pd.DataFrame(GDTSU_array, columns=GDTSL.columns, index=GDTSL.index)


In [ ]:
# rows_to_pad = DTSL.shape[0] - DTSU.shape[0]
# dummy_rows = pd.DataFrame(np.nan, index=range(rows_to_pad), columns=DTSU.columns)
# DTSU = pd.concat([DTSU, dummy_rows])

In [ ]:

# Step 4: Transfer Semi-Supervised Learning
def transfer_semi_supervised_learning(initial_model, DTSL, DTSU, GDTSL, GDTSU, n, λ, num_epochs):
    # Initialize the TextCNN classifier of the target system (TM)
    TM = tf.keras.models.clone_model(initial_model)
    TM.set_weights(initial_model.get_weights())

    # Freeze the first n layers of TM
    for layer in TM.layers[:n]:
        layer.trainable = False

    # Define the loss function and optimizer
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    optimizer = tf.keras.optimizers.Adam()

    # Generate pseudo labeled samples using the keyword avoidance perturbation mechanism
    pseudo_labeled_samples = []
    for sample in DTSL:
       # pseudo_labeled_samples.append(apply_keyword_avoidance_perturbation(sample, keyword_library))
        # pseudo_labeled_samples = np.reshape(pseudo_labeled_samples, (len(pseudo_labeled_samples), 1))
        pseudo_labeled_samples = np.reshape(pseudo_labeled_samples, (len(pseudo_labeled_samples), 7))

    # Combine the pseudo labeled and original labeled samples to form a new training set
    #combined_samples = np.concatenate((DTSL, np.array(pseudo_labeled_samples)), axis=0)
    combined_samples = np.stack((DTSL, DTSU, GDTSL, GDTSU), axis=0)
    combined_labels = np.concatenate((np.ones(len(DTSL)), np.zeros(len(pseudo_labeled_samples))), axis=0)

    # Train TM on the combined dataset
    for epoch in range(num_epochs):
        with tf.GradientTape() as tape:
            y_pred_combined = TM(combined_samples, training=True)
            loss_combined = loss_fn(combined_labels, y_pred_combined)

            # Add the consistency loss using the unlabeled samples
            # Consistency loss = λ * MSE(y_pred_combined, y_pred_unlabeled)
            # where y_pred_unlabeled is the prediction on the unlabeled dataset using TM
            y_pred_unlabeled = TM(DTSU, training=True)
            consistency_loss = λ * tf.keras.losses.MeanSquaredError()(y_pred_combined, y_pred_unlabeled)

            total_loss = loss_combined + consistency_loss

        gradients = tape.gradient(total_loss, TM.trainable_variables)
        optimizer.apply_gradients(zip(gradients, TM.trainable_variables))

    return TM

# Hyperparameters for transfer semi-supervised learning
n = 5  # Number of frozen layers
λ = 0.1  # Trade-off weight for consistency loss
num_epochs = 100  # Number of training epochs

In [ ]:
# Assuming you have defined the required data and initial_model
print(type(DTSL[0]))
print(type(DTSU))
print(type(GDTSL))
#TM = transfer_semi_supervised_learning(model, DTSL, DTSU, GDTSL, GDTSU, n, λ, num_epochs)
#TM.summary()


<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


# **Transfer Learning**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Dense, Concatenate
from tensorflow.keras.models import Model




In [ ]:
# Load and preprocess data from various sources
labelled_data = pd.read_csv('/content/labeled_data.csv', encoding='UTF-8')
unlabelled_data = pd.read_csv('/content/unlabeled_data.csv', encoding='UTF-8')
# generated_labelled_data = pd.read_csv('/content/Generated_Labelled.csv', encoding='UTF-8')
# generated_unlabelled_data = pd.read_csv('/content/Generated_Unlabelled.csv', encoding='UTF-8')



In [ ]:
# Concatenate labelled and generated labelled data
combined_labelled_data = pd.concat([labelled_data], ignore_index=True)

# Combine unlabelled and generated unlabelled data
combined_unlabelled_data = pd.concat([unlabelled_data], ignore_index=True)



In [ ]:
X_labelled = combined_labelled_data[['URL','Content-Type','Payload']]
y_labelled = combined_labelled_data['Label']

X_unlabelled = combined_unlabelled_data[['URL','Content-Type','Payload']]



In [ ]:
X_labelled = X_labelled.astype(str)
print(type(X_labelled))
print(len(X_labelled))
y_labelled = y_labelled.astype(str)
print(type(y_labelled))
print(len(y_labelled))

<class 'pandas.core.frame.DataFrame'>
23863
<class 'pandas.core.series.Series'>
23863


In [ ]:
# Tokenize and pad sequences for labelled data
max_sequence_length = 100
tokenizer = Tokenizer(filters=["'", '"', '<', '>', '+', '-', '_', '*', '=', '{', '}', '(', ')', '[', ']', '∼', '/',
                      '\\', '#', ':', ';', '?', '!', '&', '@'], oov_token='<OOV>')
tokenizer.fit_on_texts(X_labelled['Payload'])
sequences = tokenizer.texts_to_sequences(X_labelled['Payload'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
# Convert labels to one-hot encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y_labelled)
one_hot_labels = to_categorical(encoded_labels)


In [ ]:

# Split data into train and test sets for labelled data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)

# Tokenize and pad sequences for unlabelled data
sequences_unlabelled = tokenizer.texts_to_sequences(X_unlabelled['Payload'])
padded_sequences_unlabelled = pad_sequences(sequences_unlabelled, maxlen=max_sequence_length, padding='post', truncating='post')


In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences

# Assuming X_unlabelled['Payload'] contains your unlabelled data
# Replace NaN values with empty strings and convert to strings
X_unlabelled['Payload'] = X_unlabelled['Payload'].fillna('').astype(str)

# Tokenize and pad sequences for unlabelled data
sequences_unlabelled = tokenizer.texts_to_sequences(X_unlabelled['Payload'])
padded_sequences_unlabelled = pad_sequences(sequences_unlabelled, maxlen=max_sequence_length, padding='post', truncating='post')


<ipython-input-111-da5f80e2154a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_unlabelled['Payload'] = X_unlabelled['Payload'].fillna('').astype(str)


In [ ]:

# Load the pre-trained TextCNN model
pretrained_model = load_model('trained_model.h5')

# Remove the last layer (output layer) from the pre-trained model
pretrained_model.layers.pop()


In [ ]:

# Create a new output layer for the target domain
new_output_layer = Dense(len(label_encoder.classes_), activation='softmax')(pretrained_model.layers[-1].output)

# Create the transfer learning model
transfer_learning_model = Model(inputs=pretrained_model.input, outputs=new_output_layer)



In [ ]:
# Freeze the layers of the pre-trained model
for layer in transfer_learning_model.layers[:-1]:
    layer.trainable = False

# Compile the transfer learning model
transfer_learning_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the transfer learning model on labelled data
transfer_learning_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


Epoch 1/10
597/597 [==============================] - 4s 5ms/step - loss: 0.8836 - accuracy: 0.6346 - val_loss: 0.8602 - val_accuracy: 0.6354
Epoch 2/10
597/597 [==============================] - 3s 4ms/step - loss: 0.8503 - accuracy: 0.6344 - val_loss: 0.8404 - val_accuracy: 0.6352
Epoch 3/10
597/597 [==============================] - 3s 5ms/step - loss: 0.8372 - accuracy: 0.6342 - val_loss: 0.8304 - val_accuracy: 0.6352
Epoch 4/10
597/597 [==============================] - 3s 5ms/step - loss: 0.8280 - accuracy: 0.6348 - val_loss: 0.8244 - val_accuracy: 0.6352
Epoch 5/10
597/597 [==============================] - 3s 4ms/step - loss: 0.8228 - accuracy: 0.6345 - val_loss: 0.8203 - val_accuracy: 0.6352
Epoch 6/10
597/597 [==============================] - 3s 4ms/step - loss: 0.8201 - accuracy: 0.6348 - val_loss: 0.8174 - val_accuracy: 0.6352
Epoch 7/10
597/597 [==============================] - 3s 5ms/step - loss: 0.8178 - accuracy: 0.6350 - val_loss: 0.8154 - val_accuracy: 0.6352
Epoch 

In [ ]:

# Evaluate the transfer learning model on labelled data
loss, accuracy = transfer_learning_model.evaluate(X_test, y_test)
print(f'Test Loss (Labelled Data): {loss:.4f}')
print(f'Test Accuracy (Labelled Data): {accuracy:.4f}')

# Predict using the transfer learning model on unlabelled data
predictions_unlabelled = transfer_learning_model.predict(padded_sequences_unlabelled)

# Save the updated unlabelled data to a CSV file
combined_unlabelled_data['Predictions'] = predictions_unlabelled.tolist()
combined_unlabelled_data.to_csv('/content/Updated_Unlabelled.csv', index=False)

150/150 [==============================] - 0s 3ms/step - loss: 0.8117 - accuracy: 0.6352
Test Loss (Labelled Data): 0.8117
Test Accuracy (Labelled Data): 0.6352
746/746 [==============================] - 1s 2ms/step
